# Getting Started with Bayesian Optimization (NEXTorch)

This notebook accompanies the blog post on NEXTorch. You can use it to follow along and experiment with the code.

**NEXTorch** is a Python library for Bayesian Optimization designed for chemical sciences and engineering. It's built on PyTorch and BoTorch.

## Installation

If you haven't installed NEXTorch yet, run:
```bash
pip install nextorch
```

## Step 1: Define the objective function

We'll use a test function that simulates a yield optimization problem with two parameters.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nextorch import bo, doe, utils, plotting

def yield_fn(X):
    """
    Objective function that simulates a yield optimization problem.
    Takes a 2D array where each row is [xi1, xi2].
    Returns an array of yield values.
    """
    X = np.atleast_2d(X)
    results = []
    for row in X:
        xi1, xi2 = float(row[0]), float(row[1])
        x1_t, x2_t = 3 * xi1 - 15, xi2 / 50.0 - 13
        c, s = np.cos(0.5), np.sin(0.5)
        x1_r = c * x1_t - s * x2_t
        x2_r = s * x1_t + c * x2_t
        y = np.exp(-x1_r**2 / 80.0 - 0.5 * (x2_r + 0.03 * x1_r**2 - 40 * 0.03)**2)
        results.append(100.0 * y)
    return np.array(results).reshape(-1, 1)

### Visualize the objective function

Let's plot the function as a contour plot to see what we're optimizing.

In [ ]:
# Create a grid to visualize the objective function
xi1_vals = np.linspace(0, 10, 100)
xi2_vals = np.linspace(0, 1000, 100)
X1, X2 = np.meshgrid(xi1_vals, xi2_vals)
X_grid = np.column_stack((X1.ravel(), X2.ravel()))
Y_grid = yield_fn(X_grid).reshape(X1.shape)

plt.figure(figsize=(10, 8))
plt.contourf(X1, X2, Y_grid, levels=20, cmap='viridis')
plt.colorbar(label='Yield (%)')
plt.xlabel('xi1')
plt.ylabel('xi2')
plt.title('Objective Function Landscape')
plt.show()

## Step 2: Define the search space

In NEXTorch, you define parameter ranges as a list of [min, max] pairs.

In [ ]:
X_ranges = [[0, 10], [0, 1000]]
X_names = ['xi1', 'xi2']
Y_name = ['yield']
n_dim = len(X_names)

print(f"Search space: {n_dim} dimensions")
for i, (name, range_) in enumerate(zip(X_names, X_ranges)):
    print(f"  {name}: [{range_[0]}, {range_[1]}]")

## Step 3: Generate initial data with DOE

NEXTorch has built-in DOE methods. We'll use Latin Hypercube Sampling for good space-filling coverage.

In [ ]:
n_init = 3
X_init = doe.latin_hypercube(n_dim=n_dim, n_points=n_init, seed=42)

print("Initial points (unit scale [0,1]):")
print(X_init)

In [ ]:
# Convert to real scale and evaluate
X_init_real = utils.inverse_unitscale_X(X_init, X_ranges)
Y_init_real = yield_fn(X_init_real)

print("Initial points (real scale):")
for i in range(n_init):
    print(f"  Point {i+1}: xi1={X_init_real[i,0]:.2f}, xi2={X_init_real[i,1]:.2f} -> Yield={Y_init_real[i,0]:.2f}%")

## Step 4: Create the experiment and configure settings

Bundle everything into an `Experiment` object.

In [ ]:
exp = bo.Experiment("YieldOptimization")
exp.input_data(
    X_real=X_init_real,
    Y_real=Y_init_real,
    X_ranges=X_ranges,
    X_names=X_names,
    Y_names=Y_name
)
exp.set_optim_specs(
    objective_func=yield_fn,
    maximize=True
)

print("Experiment created successfully!")

## Step 5: Run the optimization loop

### Option A: Fully automated optimization

Use this when you have an objective function that can be evaluated programmatically.

In [ ]:
# Run 12 additional trials (15 total including initial 3)
exp.run_trials_auto(n_trials=12, acq_func_name='qEI')

print(f"Optimization complete! Total trials: {len(exp.Y_real)}")

### Option B: Human-in-the-loop optimization (alternative)

Use this for real lab experiments. Uncomment and run if you want to try the manual approach instead.

**Note:** Don't run this if you already ran Option A above - it would add more trials.

In [ ]:
# # Human-in-the-loop approach (uncomment to use)
# for i in range(12):
#     X_new, X_new_real, acq_func = exp.generate_next_point(
#         n_candidates=1,
#         acq_func_name='qEI'
#     )
#     print(f"Trial {i+4}: Run experiment at xi1={X_new_real[0,0]:.2f}, xi2={X_new_real[0,1]:.2f}")
#
#     # In a real lab, you would now run the experiment and measure Y_new_real
#     Y_new_real = yield_fn(X_new_real)  # Simulated here
#
#     exp.run_trial(X_new, X_new_real, Y_new_real)

## Step 6: Extract and visualize results

### Get the optimal result

In [ ]:
y_opt, X_opt, idx_opt = exp.get_optim()
print(f"Best Yield: {y_opt:.2f}%")
print(f"Optimal Parameters: xi1={X_opt[0]:.2f}, xi2={X_opt[1]:.2f}")
print(f"Found at trial: {idx_opt + 1}")

### Model performance plots

In [ ]:
# Predicted vs actual values
plotting.parity_exp(exp)
plt.show()

In [ ]:
# With 95% confidence intervals
plotting.parity_with_ci_exp(exp)
plt.show()

### Optimization progress

In [ ]:
# Discovery curve
plotting.opt_per_trial_exp(exp)
plt.show()

### Manual predictions from the surrogate model

You can query the GP model directly at any point in the design space.

In [ ]:
# Define test points
X_test = np.array([
    [5.0, 500.0],  # Center of design space
    [5.0, 700.0]   # High-yield region
])

# Get predictions with 95% confidence intervals
Y_mean, Y_lower, Y_upper = exp.predict_real(X_test, show_confidence=True)

# Ensure 1D arrays for easy iteration
Y_mean = np.ravel(Y_mean)
Y_lower = np.ravel(Y_lower)
Y_upper = np.ravel(Y_upper)

print("Manual predictions:")
for i, x_val in enumerate(X_test):
    print(f"Input: xi1={x_val[0]}, xi2={x_val[1]}")
    print(f"  Predicted Yield: {Y_mean[i]:.2f}%")
    print(f"  95% Confidence:  [{Y_lower[i]:.2f}%, {Y_upper[i]:.2f}%]")

### Custom heatmap from GP predictions

Create a custom visualization by querying the GP model across a dense grid.

In [ ]:
# Create dense grid across design space
steps = 100
xi1_range = np.linspace(X_ranges[0][0], X_ranges[0][1], steps)
xi2_range = np.linspace(X_ranges[1][0], X_ranges[1][1], steps)
X1_grid, X2_grid = np.meshgrid(xi1_range, xi2_range)

# Flatten grid for prediction
X_grid = np.column_stack((X1_grid.ravel(), X2_grid.ravel()))

# Get GP predictions
Y_pred = exp.predict_real(X_grid)
Z_pred = Y_pred.reshape(X1_grid.shape)

# Plot heatmap
plt.figure(figsize=(10, 8))
plt.imshow(
    Z_pred,
    cmap='jet',
    interpolation='gaussian',
    origin='lower',
    extent=[X_ranges[0][0], X_ranges[0][1], X_ranges[1][0], X_ranges[1][1]],
    aspect='auto'
)
plt.colorbar(label='Predicted Yield (%)')

# Overlay trial points
X_trials = np.array(exp.X_real)
plt.scatter(X_trials[:, 0], X_trials[:, 1], c='white', edgecolors='red',
            s=100, label='Trial Points')

plt.title('Design Space Heatmap (GP Model Prediction)')
plt.xlabel('xi1 (Parameter 1)')
plt.ylabel('xi2 (Parameter 2)')
plt.legend()
plt.tight_layout()
plt.show()

## Summary of results

In [ ]:
# Create a summary table of all trials
results_df = pd.DataFrame({
    'Trial': range(1, len(exp.Y_real) + 1),
    'xi1': exp.X_real[:, 0],
    'xi2': exp.X_real[:, 1],
    'Yield (%)': exp.Y_real.flatten()
})

print("All trials:")
print(results_df.to_string(index=False))
print(f"\nBest result: Trial {idx_opt + 1} with Yield = {y_opt:.2f}%")

---

## Next steps

Now that you've seen the basics, try:

1. **Change the number of initial points** (`n_init`) - more points = better initial model
2. **Try different acquisition functions** - replace `'qEI'` with `'qUCB'` or `'qPI'`
3. **Use your own objective function** - replace `yield_fn` with your real experiment
4. **Add more parameters** - extend `X_ranges` and `X_names`

For more information, check out the [NEXTorch documentation](https://nextorch.readthedocs.io/en/latest/).